# visdom

- visdom을 익히고 mnist cnn에 적용하기
- visdom은 서버를 켜서 사용해야 한다. 
- venv에서 `python -m visdom.server` 명령어를 이용해 visdom 서버를 실행 가능하다.

## visdom 사용법

- text
- image
- images
- Line plot

In [7]:
import torch
import torch.nn as nn

import torchvision
import torchvision.datasets as dsets

In [8]:
import visdom

vis = visdom.Visdom()

Setting up a new session...


In [9]:
vis.text('Hello World!', env="main")

'window_3e30b104dd1770'

In [10]:
a = torch.randn(3, 200, 200) # RGB 3 컬러로 200 x 200크기의 사진을 랜덤으로 만든다. 

vis.image(a)

'window_3e30b104e07c7e'

In [11]:
vis.images(torch.Tensor(3, 3, 28, 28))

/Users/janghee/Desktop/coding/python/.venv/lib/python3.13/site-packages/visdom/__init__.py:1402: RuntimeWarning: invalid value encountered in cast
  grid[:, h_start:h_end, w_start:w_end] = tensor[k]


'window_3e30b104e213c2'

## example (using MNIST and CIFAR10)

In [12]:
MNIST = dsets.MNIST(root="./MNIST_data", train=True, transform=torchvision.transforms.ToTensor(), download=True)
cifar10 = dsets.CIFAR10(root="./cifar10", train=True, transform=torchvision.transforms.ToTensor(), download=True)

In [13]:
data = cifar10.__getitem__(0)
print(data[0].shape)
vis.image(data[0], env='main')

torch.Size([3, 32, 32])


'window_3e30b131897e4e'

In [14]:
data = MNIST.__getitem__(0)
print(data[0].shape)
vis.image(data[0], env='main')

torch.Size([1, 28, 28])


'window_3e30b1ae448760'

In [20]:
data_loader = torch.utils.data.DataLoader(dataset=MNIST, batch_size=32, shuffle=True)

for num, value in enumerate(data_loader):
    value = value[0]

    print(value.shape)

    vis.images(value)
    break

torch.Size([32, 1, 28, 28])


## Line Plot

- x의 정확한 값을 제공하지 않으면 x는 0과 1사이의 값들로 생성된다. 

In [28]:
Y_data = torch.rand(5)
plt = vis.line(Y=Y_data)

In [29]:
X_data = torch.Tensor([1, 2, 3, 4, 5])
plt = vis.line(Y=Y_data, X=X_data)

## Line update

- 기존의 그래프를 업데이트할 수 있다. 
- win, update로 방식을 설정할 수 있다. 
    - win : 업데이트할 윈도우의 이름, 위에서 `plt`라고 정의했다.
    - update : update할 방식을 정의한다. cmd + 클릭해서 보니 `append`, `replace`, `remove`라는 파라미터로 정의할 수 있다. 

In [31]:
Y_append = torch.rand(1)
X_append = torch.Tensor([6])

vis.line(Y=Y_append, X=X_append, win=plt, update='append')

'window_3e30b397e1a870'

## Multiple Lines on Single window

In [33]:
num = torch.Tensor(list(range(0, 10)))
print(num.shape)

num = num.view(-1, 1)
print(num.shape)

num = torch.cat((num, num), dim=1)
print(num.shape)

plt = vis.line(Y=torch.rand(10, 2), X=num)

torch.Size([10])
torch.Size([10, 1])
torch.Size([10, 2])


## Line Info

In [34]:
plt = vis.line(Y_data, X_data, opts=dict(title='Test', showlegend=True))

In [35]:
plt = vis.line(Y_data, X_data, opts=dict(title='Test', legend=['1번'], showlegend=True))

In [36]:

plt = vis.line(Y=torch.rand(10, 2), X=num, opts=dict(title='Test', legend=['1번', '2번'], showlegend=True))

## mnist cnn에 visdom을 추가해서 plot을 그려보자.

In [46]:
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init

In [38]:
vis = visdom.Visdom()
vis.close()

Setting up a new session...


''

In [47]:
def loss_tracker(loss_plot, loss_value, num):
    vis.line(X=num, Y=loss_value, win=loss_plot, update='append')

In [48]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)

if device == 'cuda' : torch.cuda.manual_seed_all(777)

In [49]:
learning_rate = 0.001
training_epochs = 15
batch_size = 32

In [41]:
mnist_train = dsets.MNIST(root='MNIST_data/', train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dsets.MNIST(root='MNIST_data/', train=False, transform=transforms.ToTensor(), download=True)

In [42]:
data_loader = torch.utils.data.DataLoader(
    dataset=mnist_train,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True
)

In [54]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(3*3*128, 625, bias=True)
        
        self.relu = nn.ReLU()
        
        self.fc2 = nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        
        return out

In [50]:
model = CNN().to(device)

In [51]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

### make plot

In [56]:
loss_plt = vis.line(Y=torch.Tensor(1).zero_(), opts=dict(title='loss_tracker', legend=['loss'], showlegend=True))

In [57]:
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('Epoch:{} cost = {}'.format(epoch + 1, avg_cost))
    loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))
    
print('Learning finished')
        

Epoch:1 cost = 0.03040499985218048
Epoch:2 cost = 0.022970810532569885
Epoch:3 cost = 0.01868782937526703
Epoch:4 cost = 0.014636977575719357
Epoch:5 cost = 0.012057575397193432
Epoch:6 cost = 0.010045189410448074
Epoch:7 cost = 0.008292922750115395
Epoch:8 cost = 0.006249075755476952
Epoch:9 cost = 0.0061283535324037075
Epoch:10 cost = 0.004496397450566292
Epoch:11 cost = 0.005625160876661539
Epoch:12 cost = 0.003468252718448639
Epoch:13 cost = 0.004327105358242989
Epoch:14 cost = 0.0040261875838041306
Epoch:15 cost = 0.0030823112465441227
Learning finished


In [58]:
with torch.no_grad():
    x_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    y_test = mnist_test.test_labels.to(device)
    
    prediction = model(x_test)
    correct_prediction = torch.argmax(prediction, 1) == y_test
    
    accuracy = correct_prediction.float().mean()
    
    print('Accuracy : ', accuracy.item())

/Users/janghee/Desktop/coding/python/.venv/lib/python3.13/site-packages/torchvision/datasets/mnist.py:81: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/Users/janghee/Desktop/coding/python/.venv/lib/python3.13/site-packages/torchvision/datasets/mnist.py:71: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy :  0.9786999821662903


## clear visdom

In [27]:
vis.close(env='main')

''